<a href="https://colab.research.google.com/github/ArthurFDLR/Image_Boundaries_Analysis/blob/main/Image_Boundaries_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator

from google.colab.patches import cv2_imshow

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def draw_complex_border(border, img_shape):
    img = np.zeros(img_shape)
    for c in border:
        img[int(c.imag), int(c.real)] = 1.
    return img

def draw_border(border, img_shape):
    img = np.zeros(img_shape)
    for c in border:
        img[int(c[0]), int(c[1])] = 1.
    return img

def imshow_hist(img):
    hist,bins = np.histogram(img.flatten(),256,[0,256])

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7,3))
    ax1.imshow(img, cmap='gray'), ax1.set_xticks([]), ax1.set_yticks([])
    ax2.plot(hist/np.prod(img.shape), color = 'r', label=r'$p_r$')
    ax2.legend()
    ax2.set_xlabel(r"$r_i$")
    fig.tight_layout()
    fig.show()

def imshow(img):
    fig, ax = plt.subplots(dpi=55)
    ax.imshow(img, cmap='gray')
    ax.set_xticks([])
    ax.set_yticks([])
    fig.show()

# Global thresholding

In [ ]:
def basic_global_tresholding(img):
    delta_T = 1
    T = np.max(img) // 2 # Step 1
    T_old = T + delta_T
    while abs(T - T_old) >= delta_T: # Step 5
        img_treshold = (img > T) # Step 2
        m1, m2 = img[~img_treshold].mean(), img[img_treshold].mean() # Step 3
        T_old, T = T, (m1 + m2)//2 # Step 4
    return (img >= T)*1, T

In [ ]:
# Image import
noisy_fingerprint = cv2.imread('/content/drive/MyDrive/Colab Notebooks/ECE565/noisy_fingerprint.tif',0)

# Computation
noisy_fingerprint_tresholded, treshold = basic_global_tresholding(noisy_fingerprint)

# Visualization
fig, axs = plt.subplots(1, 3, figsize=(10,4), dpi=100)
hist,bins = np.histogram(noisy_fingerprint.flatten(),256,[0,256])
axs[0].set_title("Input image")
axs[0].imshow(noisy_fingerprint, cmap='gray'), axs[0].set_xticks([]), axs[0].set_yticks([])
axs[1].set_title("Normalized histogram")
axs[1].plot(hist/np.prod(noisy_fingerprint.shape), c='k', label=r'$p_r$')
axs[1].legend()
axs[1].set_xlabel(r"$r_i$")
axs[1].axvline(x=treshold, label='Treshold', c='r', ls='dashed')
axs[1].set_xticks(list(range(0,255,50)) + [int(treshold)])
axs[2].set_title("Global thresholding")
axs[2].imshow(noisy_fingerprint_tresholded, cmap='gray'), axs[2].set_xticks([]), axs[2].set_yticks([])
fig.tight_layout()
fig.show()

# Otsu’s thresholding

In [ ]:
def otsu_tresholding(img):
    hist,bins = np.histogram(img.flatten(),256,[0,256]) # Step 1
    hist = hist/np.prod(img.shape)
    cdf = hist.cumsum() # Step 2
    cmf = np.cumsum([p*i for (i, p) in enumerate(hist)]) # Step 3
    bcv = np.array(
        [(cmf[-1] * P - m)**2 / (P * (1. - P)) if (P * (1. - P)) != 0. else 0. \
         for P, m in zip(cdf, cmf)]
    ) # Step 4
    T = bcv.argmax().mean() # Step 5
    return 1*(img > T), T

In [ ]:
# Image import
img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/ECE565/polymersomes.tif',0)

# Computation
img_tresholded_otsu, treshold_otsu = otsu_tresholding(img)
img_tresholded, treshold = basic_global_tresholding(img)

# Visualization
fig, axs = plt.subplots(2, 2, figsize=(8,7), dpi=100)
axs = [ax for sub_ax in axs for ax in sub_ax]

hist,bins = np.histogram(img.flatten(),256,[0,256])
axs[0].set_title("Input image")
axs[0].imshow(img, cmap='gray'), axs[0].set_xticks([]), axs[0].set_yticks([])
axs[1].set_title("Normalized histogram")
axs[1].plot(hist/np.prod(noisy_fingerprint.shape), c='k', label=r'$p_r$')
axs[1].legend()
axs[1].set_xlabel(r"$r_i$")
axs[1].axvline(x=treshold, c='r', ls='dashed')
axs[1].axvline(x=treshold_otsu, c='b', ls='dashed')
axs[1].set_xticks(list(range(0,255,50)) + [int(treshold), int(treshold_otsu)])
axs[1].set_xlim(125, 225)
axs[2].set_title("Basic global thresholding")
axs[2].imshow(img_tresholded, cmap='gray'), axs[2].set_xticks([]), axs[2].set_yticks([])
axs[3].set_title("Otsu's global thresholding")
axs[3].imshow(img_tresholded_otsu, cmap='gray'), axs[3].set_xticks([]), axs[3].set_yticks([])

fig.tight_layout()
fig.show()

# Chain codes

In [ ]:
def find_upper_left(img):
    for i in range(1,img.shape[0]+1):
        for j in range(i):
            if img[i-j-1, j]:
                return (i-j-1, j)
    return None

def resample(img, grid_width, presence_threshold=0):
    img_sub = np.zeros(np.array(img.shape)//grid_width)
    img_sub_viz = np.zeros(img.shape)
    gw_half = grid_width // 2
    for x_sub, x in enumerate(range((img.shape[0]%grid_width)//2 + gw_half, img.shape[0], grid_width)):
        for y_sub, y in enumerate(range((img.shape[1]%grid_width)//2 + gw_half, img.shape[1], grid_width)):
            if img[x-gw_half:x+gw_half, y-gw_half:y+gw_half].sum() > presence_threshold:
                img_sub[x_sub,y_sub] = 1
                img_sub_viz[x-1:x+2, y-1:y+2] = np.ones((3,3))
    return img_sub, img_sub_viz

def boundary_following(border, stay_out=True, allow_diag=True):
    
    x_n = [0, -1, -1, -1,  0,  1, 1, 1] if allow_diag else [0, -1,  0, 1]
    y_n = [1,  1,  0, -1, -1, -1, 0, 1] if allow_diag else [1,  0, -1, 0]
    nbr_neighbore = len(x_n)
    angle = nbr_neighbore//2
    
    def get_neighbor(p,a):
        x, y = p[0] + x_n[a], p[1] + y_n[a]
        if (0 <= x < border.shape[0]) and (0 <= y < border.shape[1]):
            return border[x, y]
        else: return None

    b = find_upper_left(border)
    b_init = False
    coord_border = [b]
    chain_code = []

    while True:
        # Revolve around b until hit border
        while not get_neighbor(b, angle):
            angle = (angle - 1) if angle else (nbr_neighbore - 1)
        # Prefer direct neighbore
        if (not stay_out) and allow_diag and (angle%2 == 1) \
            and get_neighbor(b, (angle - 1) if angle else 7):
            angle = (angle - 1) if angle else (nbr_neighbore - 1)
        # Update b <- n(k)
        b = (b[0] + x_n[angle], b[1] + y_n[angle])
        # End condition: two successive boundary pixels already visited
        if b_init:
            if b == coord_border[1]: break
            else: b_init = False
        if b == coord_border[0]: b_init = True
        # Store new border pixel
        chain_code.append(angle)
        coord_border.append(b)
        # Reset angle, c <- n(k−1)
        angle = (angle+angle%2+2)%8 if allow_diag else (angle+1)%4
    return np.array(coord_border), chain_code

In [ ]:
# Image import
img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/ECE565/circular_stroke.tif',0)

# Computation
kernel_smoothing = np.ones((9,9),np.float32)/(9*9)
img_g = cv2.filter2D(img,-1,kernel_smoothing)
img_gB, _ = otsu_tresholding(img_g)
gB_border, _ = boundary_following(img_gB)
img_gB_border = draw_border(gB_border, img.shape)
img_gB_border_resampled_4, sampled_viz_4 = resample(img_gB_border, 50, 0)
img_gB_border_resampled_8, sampled_viz_8 = resample(img_gB_border, 50, 25)

# Visualization
fig, axs = plt.subplots(2, 3, figsize=(11,8), dpi=100)
axs = [ax for sub_ax in axs for ax in sub_ax]
for ax in axs:
    ax.set_xticks([])
    ax.set_yticks([])

axs[0].set_title('Input image')
axs[0].imshow(img, cmap='gray')
axs[1].set_title('Smoothed image')
axs[1].imshow(img_g, cmap='gray')
axs[2].set_title('Thresholded image')
axs[2].imshow(img_gB, cmap='gray')
axs[3].set_title('Outer boundary')
axs[3].imshow(img_gB_border, cmap='gray')
axs[4].set_title('Subsampled 4-directional boundary')
axs[4].imshow(sampled_viz_4, cmap='gray')
axs[5].set_title('Subsampled 8-directional boundary')
axs[5].imshow(sampled_viz_8, cmap='gray')

fig.tight_layout()
fig.show()

In [ ]:
def get_first_diff(chain_code):
    nbr_direction = max(chain_code)+1
    out = [(nbr_direction+chain_code[0]-chain_code[-1])%nbr_direction]
    for i in range(len(chain_code)-1):
        out.append((nbr_direction+chain_code[i+1]-chain_code[i])%nbr_direction)
    return out 

def get_magnitude(l):
    out = 0
    for i in l:
        out = out * 10 + i
    return out

def get_min_mag(chain_code):
    mini = get_magnitude(chain_code)
    mini_chain = chain_code
    for i in range(1,len(chain_code)):
        c = chain_code[i:] + chain_code[:i]
        mag_c = get_magnitude(c)
        if mag_c<mini:
            mini = mag_c
            mini_chain = c
    return mini_chain

In [ ]:
print('4-directional:')
_, chain_code_4 = boundary_following(img_gB_border_resampled_4, stay_out=False, allow_diag=False)
print("Chain code:\t\t", chain_code_4)

chain_code_diff_4 = get_first_diff(chain_code_4)
print("First difference:\t", chain_code_diff_4)

chain_code_diff_min_4 = get_min_mag(chain_code_diff_4)
print("Minimum magnitude:\t",chain_code_diff_min_4)

print('\n8-directional:')
_, chain_code_8 = boundary_following(img_gB_border_resampled_8, stay_out=False)
print("Chain code:\t\t", chain_code_8)

chain_code_diff_8 = get_first_diff(chain_code_8)
print("First difference:\t", chain_code_diff_8)

chain_code_diff_min_8 = get_min_mag(chain_code_diff_8)
print("Minimum magnitude:\t",chain_code_diff_min_8)

# Fourier Descriptors

In [ ]:
def fourierdescp(s):
    return np.fft.fft(s, len(s))

def ifourierdescp(a, P):
    P = min(len(a), P)
    mid = len(a) // 2
    low  = mid - P // 2
    high = mid + P // 2
    a_trunc = np.zeros(a.shape, np.complex_)
    a_trunc[low:high] = np.fft.fftshift(a)[low:high]
    return np.fft.ifft(np.fft.ifftshift(a_trunc))

In [ ]:
# Image import
img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/ECE565/chromosome.tif',0)

# Computation
border_coord, _ = boundary_following(img)
img_border = draw_border(border_coord, img.shape)
border_complex = np.array([c[1] + 1j * c[0] for c in border_coord])
fourier_descriptors = fourierdescp(border_complex)
FD_size = fourier_descriptors.shape[0]

# Vizualisation
fig, axs = plt.subplots(2, 3, figsize=(10,8), dpi=100)
axs_flat = [ax for sub_ax in axs for ax in sub_ax]
for ax in axs_flat:
    ax.set_xticks([])
    ax.set_yticks([])

axs_flat[0].set_title('Input image')
axs_flat[0].imshow(img, cmap='gray')
for i, compression in enumerate([1., .5, .1, .01, .005]):
    border_complex_smoothed = ifourierdescp(fourier_descriptors, int(FD_size*compression))
    axs_flat[i+1].set_title(r'$\gamma = ${:.1%}'.format(compression))
    axs_flat[i+1].imshow(draw_complex_border(border_complex_smoothed, img.shape), cmap='gray')

fig.tight_layout()
fig.show()